In [17]:
import os
import json
import pandas as pd

In [ ]:
name_method = {'1': '1_bart_noprompt', '2': '2_bart_topic', '3': '3_bart_length', 
               '4': '4_bart_topic_length', '5': '5_bart_contrastive_random', '6': '6_bart_contrastive_synonym',
               '7': '7_bart_contrastive_combine', '8': '8_bart_contrastive_combine_word_tagger', '9': '9_bart-contrastive-combine-prompt-tagger'}

In [ ]:
total_method = 4

In [32]:
def load_gold_summary(file_path):
    """load result jsonl data"""

    data = []

    with open(file_path, "r") as f:
        for line in f:
            data.append(json.loads(line))

    id_list = [sample["fname"] for sample in data]

    summary_list1 = [sample["summary1"] for sample in data]
    summary_list2 = [sample["summary2"] for sample in data]
    summary_list3 = [sample["summary3"] for sample in data]

    data_dict = {
    "fname": id_list,
    "gen_summary1": summary_list1,
    "gen_summary2": summary_list2,
    "gen_summary3": summary_list3,
    }

    return data_dict

In [26]:
def load_result(file_path):
    """load result jsonl data"""

    data = []

    with open(file_path, "r") as f:
        for line in f:
            data.append(json.loads(line))

    id_list = [sample["fname"] for sample in data]

    summary_list1 = [sample["gen_summary1"] for sample in data]
    summary_list2 = [sample["gen_summary2"] for sample in data]
    summary_list3 = [sample["gen_summary3"] for sample in data]

    data_dict = {
    "fname": id_list,
    "gen_summary1": summary_list1,
    "gen_summary2": summary_list2,
    "gen_summary3": summary_list3,
    }

    return data_dict

In [33]:
gold_summary = load_gold_summary(f"./data/dialogsum/dialogsum.test.jsonl")

In [34]:
gold_summary_df = pd.DataFrame.from_dict(gold_summary)

In [35]:
gold_summary_df.head()

,fname,gen_summary1,gen_summary2,gen_summary3
0,test_0,Ms. Dawson helps #Person1# to write a memo to ...,In order to prevent employees from wasting tim...,Ms. Dawson takes a dictation for #Person1# abo...
1,test_1,#Person2# arrives late because of traffic jam....,#Person2# decides to follow #Person1#'s sugges...,#Person2# complains to #Person1# about the tra...
2,test_2,#Person1# tells Kate that Masha and Hero get d...,#Person1# tells Kate that Masha and Hero are g...,#Person1# and Kate talk about the divorce betw...
3,test_3,#Person1# and Brian are at the birthday party ...,#Person1# attends Brian's birthday party. Bria...,#Person1# has a dance with Brian at Brian's bi...
4,test_4,#Person1# is surprised at the Olympic Stadium'...,#Person2# shows #Person1# around the construct...,#Person2# introduces the Olympic Stadium's fin...


In [27]:
list_method = list(range(1 ,total_method + 1))
result_all_df = {}
for i in list_method:
    result_dict = load_result(f"./result/{i}.jsonl")
    result_df = pd.DataFrame.from_dict(result_dict)
    result_all_df[i] = result_df

In [28]:
result_all_df[2].head()

,fname,gen_summary1,gen_summary2,gen_summary3
0,test_0,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
1,test_1,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic again. #Person1...
2,test_2,#Person1# tells Kate Masha and Hero are gettin...,#Person1# tells Kate Masha and Hero are gettin...,#Person1# tells Kate Masha and Hero are gettin...
3,test_3,#Person1# comes to Brian's birthday party and ...,#Person1# comes to Brian's birthday party and ...,#Person1# comes to Brian's birthday party and ...
4,test_4,#Person1# thinks the Olympic park is big. #Per...,#Person1# thinks the Olympic park is big. #Per...,#Person1# thinks the Olympic park is big. #Per...


In [29]:
import difflib
import nltk
import re

%matplotlib inline

In [30]:
'''
Find the matching substrings in 2 strings.
:parameter
    :param a: string - raw text
    :param b: string - raw text
:return
    2 lists used in to display matches
'''
def utils_split_sentences(a, b):
    ## find clean matches
    match = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=True)
    lst_match = [block for block in match.get_matching_blocks() if block.size > 20]

    ## difflib didn't find any match
    if len(lst_match) == 0:
        lst_a, lst_b = nltk.sent_tokenize(a), nltk.sent_tokenize(b)

    ## work with matches
    else:
        first_m, last_m = lst_match[0], lst_match[-1]

        ### a
        string = a[0 : first_m.a]
        lst_a = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = a[m.a : m.a+m.size]
            lst_a.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = a[m.a+m.size : next_m.a]
                lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = a[last_m.a+last_m.size :]
        lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]

        ### b
        string = b[0 : first_m.b]
        lst_b = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = b[m.b : m.b+m.size]
            lst_b.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = b[m.b+m.size : next_m.b]
                lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = b[last_m.b+last_m.size :]
        lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]

    return lst_a, lst_b


'''
Highlights the matched strings in text.
:parameter
    :param a: string - raw text
    :param b: string - raw text
    :param both: bool - search a in b and, if True, viceversa
    :param sentences: bool - if False matches single words
:return
    text html, it can be visualized on notebook with display(HTML(text))
'''
def display_string_matching(a, b, both=True, sentences=True, titles=[]):
    if sentences is True:
        lst_a, lst_b = utils_split_sentences(a, b)
    else:
        lst_a, lst_b = a.split(), b.split()

    ## highlight a
    first_text = []
    for i in lst_a:
        if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_b]:
            first_text.append('<span style="background-color:rgba(255,215,0,0.1);">' + i + '</span>')
        else:
            first_text.append(i)
    first_text = ' '.join(first_text)

    ## highlight b
    second_text = []
    if both is True:
        for i in lst_b:
            if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_a]:
                second_text.append('<span style="background-color:rgba(255,215,0,0.1);">' + i + '</span>')
            else:
                second_text.append(i)
    else:
        second_text.append(b)
    second_text = ' '.join(second_text)

    ## concatenate
    if len(titles) > 0:
        first_text = "<strong>"+titles[0]+"</strong><br>"+first_text
    if len(titles) > 1:
        second_text = "<strong>"+titles[1]+"</strong><br>"+second_text
    else:
        second_text = "---"*65+"<br><br>"+second_text
    final_text = first_text +'<br><br>'+ second_text
    return final_text

In [31]:
list_method = list(range(1 ,total_method + 1))
result_all_df = {}
for i in list_method:
    print(i)

1
2
3
4


In [ ]:
match = display_string_matching(text_a, text_b, both=True, sentences=False, titles=["Full Text", "Actual Text"])
from IPython.core.display import display, HTML
display(HTML(match))